In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(root="data", train=True,
    download=True, transform=ToTensor())
test_data = datasets.FashionMNIST(root="data", train=False,
    download=True, transform=ToTensor())

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
  def __init__(self, input_size, layers_data: list, num_classes):
    super(NeuralNetwork, self).__init__()
    self.flatten = nn.Flatten()
    self.layers = nn.ModuleList()
    self.input_size = input_size
    for output_size, activation_function in layers_data:
      self.layers.append(nn.Linear(input_size, output_size))
      input_size = output_size
      self.layers.append(activation_function)
    self.layers.append(nn.Linear(input_size, num_classes))
  def forward(self, x):
    x = self.flatten(x)
    for layer in self.layers:
      x = layer(x)
    return x

torch.manual_seed(23)
C = 10
N,H,W = training_data.data.shape; D = H*W
M1, M2 = 512, 512
model = NeuralNetwork(D, [(M1, nn.ReLU()), (M2, nn.ReLU())], C)

def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        pred = model(X); loss = loss_fn(pred, y)
        optimizer.zero_grad(); loss.backward(); optimizer.step() # backprop
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"trloss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset); nbatches = len(dataloader)
    teloss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X); teloss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    teloss /= nbatches; correct /= size
    print(f"teacc: {(100*correct):>0.1f}%, teloss: {teloss:>8f} \n")

learning_rate = 1e-3
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
trloss: 2.311403  [    0/60000]
trloss: 0.584715  [ 6400/60000]
trloss: 0.425315  [12800/60000]
trloss: 0.523651  [19200/60000]
trloss: 0.457378  [25600/60000]
trloss: 0.440629  [32000/60000]
trloss: 0.386600  [38400/60000]
trloss: 0.524252  [44800/60000]
trloss: 0.461581  [51200/60000]
trloss: 0.496389  [57600/60000]
teacc: 84.0%, teloss: 0.429711 

Epoch 2
-------------------------------
trloss: 0.265165  [    0/60000]
trloss: 0.350779  [ 6400/60000]
trloss: 0.286453  [12800/60000]
trloss: 0.386790  [19200/60000]
trloss: 0.422265  [25600/60000]
trloss: 0.377476  [32000/60000]
trloss: 0.321750  [38400/60000]
trloss: 0.494972  [44800/60000]
trloss: 0.361353  [51200/60000]
trloss: 0.428331  [57600/60000]
teacc: 85.0%, teloss: 0.405380 

Epoch 3
-------------------------------
trloss: 0.220827  [    0/60000]
trloss: 0.326628  [ 6400/60000]
trloss: 0.243786  [12800/60000]
trloss: 0.318946  [19200/60000]
trloss: 0.421630  [25600/60000]
trloss: 0.3500